In [133]:
import sys
sys.path.append('C:\\Anaconda2\\Lib\\site-packages\\blpfunctions')
from blpfunctions import blpfunctions as blp
#import blpfunctions as blp
import datetime as dt
import pandas as pd
from pandas.tseries.offsets import *
from holidays_jp import CountryHolidays
import dateutil.rrule as RR
import sys
import csv
import numpy as np
import time
import gc
%matplotlib inline


In [134]:
def hist_stock_volcurve(ind, event, edate, numdays, interval,fld_lst):
    #sec_list = blp.get_index(ind)
    sec_list = ind
    volcurves = pd.DataFrame()
    fmt = "%Y-%m-%d" + 'T' + "%H:%M:%S"  #Assumes no milliseconds
    endDateTime = dt.datetime.strptime(edate, fmt)
    #skip SQ and holidays (Actually turns out cannot skip SQ days in blp call)
    day1 = dt.datetime(int(edate[0:4]),1,1)
    sq = list(RR.rrule(RR.MONTHLY,byweekday=RR.FR,bysetpos=2,dtstart=day1,until=endDateTime))
    hols = list(zip(*CountryHolidays.get('JP', int(edate[0:4])))[0])
    skipdays = hols #+ sq if want to skip SQ days
    bday_jp = CustomBusinessDay(holidays=skipdays)
    
    startDateTime = endDateTime.replace(hour=9) - numdays*bday_jp
    numdays = pd.date_range(startDateTime, endDateTime, freq=bday_jp).nunique()
    sdate = startDateTime.strftime(fmt)
    
    for stock in sec_list:
        output=blp.get_Bars(stock, event, sdate, edate, interval, fld_lst)
        output.rename(columns={'VOLUME':stock},inplace=True)
        volcurves = volcurves.join(output,how="outer")

    #process the raw data into historical averages
    volcurves.rename(columns=lambda x: x[:4], inplace=True)
    timevect = pd.Series(volcurves.index.values)
    timeframet = timevect.to_frame()
    timeframet.columns = ['date']
    timeframet.set_index(timevect,inplace=True)
    timeframet['bucket'] = timeframet['date'].apply(lambda x: dt.datetime.strftime(x, '%H:%M:%S'))
    timeframet['date'] = timeframet['date'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
    timeframet=timeframet.join(volcurves)
    timeframet = timeframet[timeframet.bucket != '15:10:00']
    absvolcurve = pd.pivot_table(timeframet,index='bucket',columns='date').fillna(0).T.reset_index(level=0,drop=True).iloc[:,:60].cumsum(axis=1)
    
    volcurve = absvolcurve.div(absvolcurve.sum(axis=1), axis=0).cumsum(axis=1)
            
    return absvolcurve.fillna(method='bfill'), volcurve.fillna(method='bfill')

In [135]:
def load_data(stock, seq_len, tt_split=0.1):
    num_features = len(stock.columns) # 5
    data = stock.as_matrix() 
    sequence_length = seq_len + 1 # index starting from 0
    result = []
    
    for index in range(len(data) - sequence_length): # maxmimum date = lastest date - sequence length
        result.append(data[index: index + sequence_length]) # index : index + seq_len
    
    result = np.array(result)
    row = round((1-tt_split) * result.shape[0]) # default 90% split
    train = result[:int(row), :] # 90% date, all features 
    
    x_train = train[:, :-1] 
    y_train = train[:, :-1][:,-1]
    
    x_test = result[int(row):, :-1] 
    y_test = result[int(row):, :-1][:,-1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], num_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], num_features))  

    return [x_train, y_train, x_test, y_test]

In [136]:
def proc_data(stock_list, window):      
    fld_l = ["VOLUME"]
    event_l = ["TRADE"]
    iv = 5
    ed1 = "2017-09-28T15:00:00"
    days = 57
    Train_list = []
    test_list =[]
    y_dict = {}
    y_test_dict = {}
    for ind,stock in enumerate(stock_list):
        #abscurve, relcurve = hist_stock_volcurve([stock +' JT Equity'],event_l,ed1,days,iv,fld_l)
        abscurve, relcurve = hist_stock_volcurve([stock],event_l,ed1,days,iv,fld_l)
        X_train, y_train, X_test, y_test = load_data(relcurve, window)
        aX_train, ay_train, aX_test, ay_test = load_data(abscurve, window)
        Train_list.append(X_train)
        Train_list.append(aX_train)
        y_dict[stock] = y_train
        #y_list.append(ay_train)
        test_list.append(X_test)  
        test_list.append(aX_test) 
        y_test_dict[stock]=y_test
        #y_test_list.append(ay_test)
    return Train_list, y_dict, test_list, y_test_dict

In [137]:
from keras.models import Model
from keras.layers import Dense, Activation, Input, LSTM, Dropout, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint


early_stopping = EarlyStopping(monitor='val_loss', patience=20)
SINGLE_ATTENTION_VECTOR = False

def attention_3d_block(inputs,num_lstm,time_steps):
    #inputs.shape = (batch_size, time_steps, input_dim)
    #input_dim = int(inputs.shape[2])
    input_dim = num_lstm
    TIME_STEPS=time_steps
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul

def build_BDLSTM_model(layers):
    d = 0.2
    num_lstm = 1024
      
    shared_lstm = Bidirectional(LSTM(num_lstm, dropout=d, recurrent_dropout=d,return_sequences=False))
    #shared_lstm = LSTM(num_lstm, dropout=d, recurrent_dropout=d)
    #shared_lstm2 = Bidirectional(LSTM(num_lstm, dropout=d, recurrent_dropout=d))
    
    s1 = Input(shape=(layers[1], layers[0]))
    s2 = Input(shape=(layers[1], layers[0]))
    
    x1 = shared_lstm(s1)
    #x1 = BatchNormalization()(x1)
    x2 = shared_lstm(s2)
    x2 = BatchNormalization()(x2)
    
#     x1 = shared_lstm2(x1)
#     x2 = shared_lstm2(x2)
    
    merged = concatenate([x1,x2])
    
    merged = Dense(128, kernel_initializer="uniform", activation='relu')(merged)
    merged = Dropout(0.6)(merged)
    
    merged = Dense(128, kernel_initializer="uniform", activation='relu')(merged)
    merged = Dropout(0.6)(merged)
                                
    merged = Dense(128, kernel_initializer="uniform", activation='relu')(merged)
    merged = Dropout(0.6)(merged)
    
    preds = Dense(layers[0], kernel_initializer="uniform", activation='linear')(merged)
        
    start = time.time()
    model = Model(inputs = [s1,s2], outputs = [preds])
    model.compile(loss='mse',optimizer='nadam', metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

def BDLSTM_model_score(model, X_train, y_train, aX_train, aX_test, X_test, y_test):
    trainScore = model.evaluate([X_train,aX_train], y_train ,batch_size=50, verbose=0)
    #print('Train Score: %.5f MSE (%.4f RMSE)' % (trainScore[0], np.sqrt(trainScore[0])))

    testScore = model.evaluate([X_test, aX_test], y_test, batch_size=50, verbose=0)
    #print('Test Score: %.5f MSE (%.4f RMSE)' % (testScore[0], np.sqrt(testScore[0])))
    return trainScore[0], testScore[0]

In [49]:
abscurve

bucket,09:00:00,09:05:00,09:10:00,09:15:00,09:20:00,09:25:00,09:30:00,09:35:00,09:40:00,09:45:00,...,14:05:00,14:10:00,14:15:00,14:20:00,14:25:00,14:30:00,14:35:00,14:40:00,14:45:00,14:50:00
date,,,,,,,,,,,,,,,,,,,,,
2017-09-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1144000.0,1262000.0,1369000.0,1467000.0,1629000.0,1813000.0,1941000.0,2178000.0,2366000.0,2722000.0
2017-09-28,2183000.0,2726000.0,3002000.0,3275000.0,3419000.0,3580000.0,3717000.0,3887000.0,4001000.0,4083000.0,...,10120000.0,10212000.0,10338000.0,10647000.0,10930000.0,11119000.0,11464000.0,11728000.0,12141000.0,12627000.0
2017-09-29,1014000.0,1470000.0,1621000.0,1877000.0,2036000.0,2124000.0,2284000.0,2517000.0,2679000.0,2776000.0,...,7268000.0,7268000.0,7268000.0,7268000.0,7268000.0,7268000.0,7268000.0,7268000.0,7268000.0,7268000.0


In [46]:
window = 3
fld_l = ["VOLUME"]
event_l = ["TRADE"]
iv = 5
ed1 = "2017-09-28T15:00:00"
days = 60

abscurve, relcurve = hist_stock_volcurve(['6501 JT Equity'],event_l,ed1,days,iv,fld_l)
X_train, y_train, X_test, y_test = load_data(relcurve, window)
aX_train, ay_train, aX_test, ay_test = load_data(abscurve, window)

#model = build_model([feats,window,1])
#model.fit([X_train, aX_train], y_train, batch_size=5, epochs=200, validation_split=0.1, verbose=1)

['6501 JT Equity']


IndexError: too many indices for array

In [ ]:
# print(X_test[-1])
#diff=[]
#ratio=[]
p = model.predict([X_test, aX_test],batch_size=4096)
print (p.shape)
# for each data index in test data
# for u in range(len(y_test)):
#     # pr = prediction day u
#     pr = p[u][0]
#     # (y_test day u / pr) - 1
#     ratio.append((y_test[u]/pr)-1)
#     diff.append(abs(y_test[u]- pr))
#     # print(u, y_test[u], pr, (y_test[u]/pr)-1, abs(y_test[u]- pr))
#     # Last day prediction
#     # print(p[-1]) 

In [ ]:
import matplotlib.pyplot as plt2
p = model.predict([X_test, aX_test],batch_size=4096)
plt2.plot(p[4],color='red', label='Prediction')
plt2.plot(y_test[4],color='blue', label='Actual')
plt2.legend(loc='best')
plt2.show()

In [ ]:
X_train.shape

In [ ]:
#Build, Train, and Save results per Cluster

In [110]:
#Cluster Cross-Stock Bidirectional LSTM
import keras.backend as K
from keras.models import Model, Input
from keras.layers import Activation, Input, LSTM, Dropout, Bidirectional, Lambda
from keras.layers.core import Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=20)

def build_cluster_cross_stock_model(layers):
    d = 0.1
    num_lstm = 256
    shape = [layers[1], layers[0]]
    
    lstm = Bidirectional(LSTM(num_lstm, droupout=d, recurrent_dropout=d))
    #lstm = LSTM(num_lstm, dropout=d)
    batchnorm = BatchNormalization()
    
    input_list = []
    for num in range(0,2*layers[2]):
        input_list.append(Input(shape=shape))
    print('Using {} tensors as input.'.format(len(input_list)))    
        
    lstm_layers=[]
    for i in range(0,layers[2]):
        lstm_layers.append(lstm)
    print('Using {} interlaced LSTM layers on those inputs.'.format(len(lstm_layers)))
    
    lstm_outputs=[]
    for i in range(1,2*layers[2],2):
        id_lstm_layer = i//2
        shared_lstm = lstm_layers[id_lstm_layer]
        lstm_outputs.append(shared_lstm(input_list[i - 1]))
        lstm_outputs.append(batchnorm(shared_lstm(input_list[i])))
        print('LSTM layer id = {} is linked to inputs whose ids are {} and {}.'.format(id_lstm_layer, i - 1, i))
                            
    merged = Lambda(lambda x: K.stack(x,axis=1),output_shape=(2*layers[2], 2*num_lstm))(lstm_outputs)
    merged = Flatten()(merged)                        
    merged = Dense(128, activation='relu')(merged)
    merged = Dropout(0.7)(merged)
    merged = Dense(128, activation='relu')(merged)
    merged = Dropout(0.7)(merged)
    preds = Dense(layers[0], activation='linear')(merged)
    
    start = time.time()
    model = Model(inputs = input_list, outputs = [preds])
    model.compile(loss='mse',optimizer='nadam', metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

def cross_stock_model_score(model, Train, y, test, test_y):
    trainScore = model.evaluate(Train, y ,batch_size=50, verbose=0)
    #print('Train Score: %.5f MSE (%.4f RMSE)' % (trainScore[0], np.sqrt(trainScore[0])))

    testScore = model.evaluate(test, test_y, batch_size=50, verbose=0)
    #print('Test Score: %.5f MSE (%.4f RMSE)' % (testScore[0], np.sqrt(testScore[0])))
    return trainScore[0], testScore[0]


In [128]:
def go(clust,mode):
    
    window = 5
    feats = 60
    
    print('Training in %s mode' % mode)
    #stock_list = asses.item()[clust]
    stock_list = blp.get_index(clust)

    output = pd.DataFrame(columns=['stock','bst_val_score','train_rmse','test_rmse'],index=stock_list)
    output['stock'] = stock_list
    num_stocks = len(stock_list)
    #tb = TensorBoard(log_dir='./tblogs', histogram_freq=0, write_graph=True, write_grads=False, write_images=False)
    
    Train_list, y_dict, test_list, y_test_dict = proc_data(stock_list, window)
    
    if mode == 'cross':
        model = build_cluster_cross_stock_model([feats,window,num_stocks])
        STAMP = 'Vert_Cross-stock-cluster-' + str(clust)
        bst_model_path = STAMP + '.h5'
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        
    elif mode == 'single':
        model = build_BDLSTM_model([feats,window,num_stocks])
        STAMP = 'Vert_BDLSTM_768_0_60-cluster-' + str(clust)
        bst_model_path = STAMP + '.h5'
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
     
    for ind,stock in enumerate(stock_list): 
        if mode == 'cross':
            model.load_weights(bst_model_path)
            hist = model.fit(Train_list, y_dict[stock], batch_size=5, epochs=100, validation_split=0.1, verbose=True,
                  callbacks=[early_stopping])
            bst_val_score=min(hist.history['val_loss'])
            #p = model.predict(test_list,batch_size=50)
            train_mse, test_mse = cross_stock_model_score(model, Train_list, y_dict[stock], test_list, y_test_dict[stock])
            
        elif mode == 'single':
            X_train = Train_list[2*ind]
            aX_train = Train_list[(2*ind)+1]           
            X_test = test_list[2*ind]
            aX_test = test_list[(2*ind)+1]
            y_train = y_dict[stock]
            y_test = y_test_dict[stock]
            #model.load_weights(bst_model_path)
            hist = model.fit([X_train, aX_train], y_train, batch_size=5, epochs=100, validation_split=0.1, verbose=True,
                  callbacks=[early_stopping])
            bst_val_score=min(hist.history['val_loss'])
            #p = model.predict([X_test,aX_test],batch_size=50)
            train_mse, test_mse = BDLSTM_model_score(model, X_train, y_train, aX_train, aX_test, X_test, y_test)     
      
        output['bst_val_score'][stock] = bst_val_score
        output['train_rmse'][stock] = np.sqrt(train_mse)
        output['test_rmse'][stock] = np.sqrt(test_mse)
    
        print('Stock %s: Best Val Score: %.4f, Train Score: %.4f RMSE, Test Score %.4f RMSE' 
              % (stock[:4], bst_val_score, np.sqrt(train_mse), np.sqrt(test_mse)))
        
    model.save_weights(bst_model_path)
    output.to_csv(STAMP + '-Vertical.csv.gz', index=False, compression='gzip')

In [131]:
def run(clusters):
    #test on smallest cluster
    #clusters = [4]
    mode = 'single'
    #mode = 'cross'
    
#     for clust in clusters:
#         print 'cluster:', clust
#         go(clust,mode)
    go('TPX500 Index',mode)
    #go('TPXC30 Index',mode)

In [ ]:
if __name__ == '__main__':
    asses = np.load('clusters.npy')
    clusters = asses.item().keys()
    run(clusters)


Training in single mode
('Compilation Time : ', 0.006999969482421875)
Train on 42 samples, validate on 5 samples
Epoch 1/100
42/42 [==============================] - 16s - loss: 0.1779 - acc: 0.0476 - val_loss: 0.1111 - val_acc: 0.0000e+00
Epoch 2/100
42/42 [==============================] - 17s - loss: 0.0953 - acc: 0.0000e+00 - val_loss: 0.1210 - val_acc: 0.0000e+00
Epoch 3/100
42/42 [==============================] - 16s - loss: 0.0623 - acc: 0.1905 - val_loss: 0.0554 - val_acc: 1.0000
Epoch 4/100
42/42 [==============================] - 15s - loss: 0.0456 - acc: 0.3333 - val_loss: 0.0240 - val_acc: 1.0000
Epoch 5/100
42/42 [==============================] - 15s - loss: 0.0465 - acc: 0.3095 - val_loss: 0.0320 - val_acc: 1.0000
Epoch 6/100
42/42 [==============================] - 15s - loss: 0.0438 - acc: 0.4762 - val_loss: 0.0436 - val_acc: 1.0000
Epoch 7/100
42/42 [==============================] - 14s - loss: 0.0355 - acc: 0.3810 - val_loss: 0.0335 - val_acc: 1.0000
Epoch 8/100
42

42/42 [==============================] - 15s - loss: 0.0029 - acc: 0.8810 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 9/100
42/42 [==============================] - 16s - loss: 0.0028 - acc: 0.7381 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 10/100
42/42 [==============================] - 16s - loss: 0.0029 - acc: 0.8810 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 11/100
42/42 [==============================] - 16s - loss: 0.0031 - acc: 0.8810 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 12/100
42/42 [==============================] - 15s - loss: 0.0025 - acc: 0.8810 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 13/100
42/42 [==============================] - 15s - loss: 0.0035 - acc: 0.8810 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 14/100
42/42 [==============================] - 15s - loss: 0.0030 - acc: 0.9524 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 15/100
42/42 [==============================] - 16s - loss: 0.0030 - acc: 0.8810 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 16/100
42/42 [==

42/42 [==============================] - 17s - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 74/100
42/42 [==============================] - 17s - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 75/100
42/42 [==============================] - 17s - loss: 0.0022 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 1.0000
Stock 1662: Best Val Score: 0.0010, Train Score: 0.0229 RMSE, Test Score 0.0234 RMSE
Train on 42 samples, validate on 5 samples
Epoch 1/100
42/42 [==============================] - 16s - loss: 0.0021 - acc: 1.0000 - val_loss: 2.0012e-04 - val_acc: 1.0000
Epoch 2/100
42/42 [==============================] - 16s - loss: 0.0023 - acc: 1.0000 - val_loss: 2.3512e-04 - val_acc: 1.0000
Epoch 3/100
42/42 [==============================] - 16s - loss: 0.0021 - acc: 1.0000 - val_loss: 1.8417e-04 - val_acc: 1.0000
Epoch 4/100
42/42 [==============================] - 16s - loss: 0.0020 - acc: 1.0000 - val_loss: 2.6250e-04 - val_acc: 1.0000
Epoch

42/42 [==============================] - 17s - loss: 0.0025 - acc: 1.0000 - val_loss: 3.7797e-04 - val_acc: 1.0000
Epoch 22/100
42/42 [==============================] - 17s - loss: 0.0015 - acc: 1.0000 - val_loss: 3.6852e-04 - val_acc: 1.0000
Epoch 23/100
42/42 [==============================] - 16s - loss: 0.0016 - acc: 1.0000 - val_loss: 3.3510e-04 - val_acc: 1.0000
Epoch 24/100
42/42 [==============================] - 17s - loss: 0.0019 - acc: 1.0000 - val_loss: 4.7945e-04 - val_acc: 1.0000
Epoch 25/100
42/42 [==============================] - 16s - loss: 0.0020 - acc: 1.0000 - val_loss: 3.3725e-04 - val_acc: 1.0000
Epoch 26/100
42/42 [==============================] - 17s - loss: 0.0019 - acc: 1.0000 - val_loss: 3.7727e-04 - val_acc: 1.0000
Epoch 27/100
42/42 [==============================] - 18s - loss: 0.0015 - acc: 1.0000 - val_loss: 4.1319e-04 - val_acc: 1.0000
Epoch 28/100
42/42 [==============================] - 18s - loss: 0.0018 - acc: 1.0000 - val_loss: 5.2696e-04 - val_a

42/42 [==============================] - 17s - loss: 0.0018 - acc: 1.0000 - val_loss: 5.8419e-04 - val_acc: 1.0000
Epoch 47/100
42/42 [==============================] - 17s - loss: 0.0017 - acc: 1.0000 - val_loss: 5.6020e-04 - val_acc: 1.0000
Epoch 48/100
42/42 [==============================] - 17s - loss: 0.0017 - acc: 1.0000 - val_loss: 5.6917e-04 - val_acc: 1.0000
Epoch 49/100
42/42 [==============================] - 16s - loss: 0.0017 - acc: 1.0000 - val_loss: 5.8834e-04 - val_acc: 1.0000
Epoch 50/100
42/42 [==============================] - 17s - loss: 0.0019 - acc: 1.0000 - val_loss: 6.2747e-04 - val_acc: 1.0000
Epoch 51/100
42/42 [==============================] - 16s - loss: 0.0014 - acc: 1.0000 - val_loss: 5.9806e-04 - val_acc: 1.0000
Stock 1802: Best Val Score: 0.0005, Train Score: 0.0329 RMSE, Test Score 0.0125 RMSE
Train on 42 samples, validate on 5 samples
Epoch 1/100
42/42 [==============================] - 17s - loss: 0.0018 - acc: 1.0000 - val_loss: 3.1437e-04 - val_ac

...

42/42 [==============================] - 38s - loss: 7.5314e-04 - acc: 1.0000 - val_loss: 6.7330e-04 - val_acc: 1.0000
Epoch 17/100
42/42 [==============================] - 39s - loss: 7.5688e-04 - acc: 1.0000 - val_loss: 6.9581e-04 - val_acc: 1.0000
Epoch 18/100
42/42 [==============================] - 39s - loss: 7.5163e-04 - acc: 1.0000 - val_loss: 7.0846e-04 - val_acc: 1.0000
Epoch 19/100
42/42 [==============================] - 39s - loss: 7.5318e-04 - acc: 1.0000 - val_loss: 7.0494e-04 - val_acc: 1.0000
Epoch 20/100
42/42 [==============================] - 39s - loss: 7.5625e-04 - acc: 1.0000 - val_loss: 7.1356e-04 - val_acc: 1.0000
Epoch 21/100
42/42 [==============================] - 38s - loss: 7.5270e-04 - acc: 1.0000 - val_loss: 7.5840e-04 - val_acc: 1.0000
Epoch 22/100
42/42 [==============================] - 39s - loss: 7.5565e-04 - acc: 1.0000 - val_loss: 7.7915e-04 - val_acc: 1.0000
Stock 9783: Best Val Score: 0.0006, Train Score: 0.0275 RMSE, Test Score 0.0206 RMSE
Trai

42/42 [==============================] - 38s - loss: 2.7637e-04 - acc: 1.0000 - val_loss: 5.7253e-04 - val_acc: 1.0000
Epoch 11/100
42/42 [==============================] - 39s - loss: 2.7473e-04 - acc: 1.0000 - val_loss: 5.5438e-04 - val_acc: 1.0000
Epoch 12/100
42/42 [==============================] - 39s - loss: 2.7621e-04 - acc: 1.0000 - val_loss: 5.3450e-04 - val_acc: 1.0000
Epoch 13/100
42/42 [==============================] - 38s - loss: 2.7612e-04 - acc: 1.0000 - val_loss: 5.3909e-04 - val_acc: 1.0000
Epoch 14/100
42/42 [==============================] - 39s - loss: 2.7866e-04 - acc: 1.0000 - val_loss: 5.6411e-04 - val_acc: 1.0000
Epoch 15/100
42/42 [==============================] - 38s - loss: 2.7541e-04 - acc: 1.0000 - val_loss: 5.7731e-04 - val_acc: 1.0000
Epoch 16/100
42/42 [==============================] - 38s - loss: 2.8132e-04 - acc: 1.0000 - val_loss: 5.6615e-04 - val_acc: 1.0000
Epoch 17/100
42/42 [==============================] - 38s - loss: 2.7707e-04 - acc: 1.000

42/42 [==============================] - 38s - loss: 9.4545e-04 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 1.0000
Stock 9861: Best Val Score: 0.0008, Train Score: 0.0314 RMSE, Test Score 0.0169 RMSE
Train on 42 samples, validate on 5 samples
Epoch 1/100
42/42 [==============================] - 38s - loss: 0.0025 - acc: 1.0000 - val_loss: 3.1433e-04 - val_acc: 1.0000
Epoch 2/100
42/42 [==============================] - 38s - loss: 0.0013 - acc: 1.0000 - val_loss: 6.7980e-04 - val_acc: 1.0000
Epoch 3/100
42/42 [==============================] - 38s - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 4/100
42/42 [==============================] - 39s - loss: 0.0012 - acc: 1.0000 - val_loss: 9.9500e-04 - val_acc: 1.0000
Epoch 5/100
42/42 [==============================] - 38s - loss: 0.0012 - acc: 1.0000 - val_loss: 8.0063e-04 - val_acc: 1.0000
Epoch 6/100
42/42 [==============================] - 39s - loss: 0.0011 - acc: 1.0000 - val_loss: 7.2536e-04 - val_acc: 1.0000

42/42 [==============================] - 38s - loss: 6.2329e-04 - acc: 1.0000 - val_loss: 5.9420e-04 - val_acc: 1.0000
Epoch 14/100
42/42 [==============================] - 38s - loss: 6.2255e-04 - acc: 1.0000 - val_loss: 5.8349e-04 - val_acc: 1.0000
Epoch 15/100
42/42 [==============================] - 37s - loss: 6.1829e-04 - acc: 1.0000 - val_loss: 5.5402e-04 - val_acc: 1.0000
Epoch 16/100
42/42 [==============================] - 38s - loss: 6.3819e-04 - acc: 1.0000 - val_loss: 5.3604e-04 - val_acc: 1.0000
Epoch 17/100
42/42 [==============================] - 38s - loss: 6.2600e-04 - acc: 1.0000 - val_loss: 5.7430e-04 - val_acc: 1.0000
Epoch 18/100
42/42 [==============================] - 38s - loss: 6.2240e-04 - acc: 1.0000 - val_loss: 5.8950e-04 - val_acc: 1.0000
Epoch 19/100
42/42 [==============================] - 38s - loss: 6.2210e-04 - acc: 1.0000 - val_loss: 5.8205e-04 - val_acc: 1.0000
Epoch 20/100
42/42 [==============================] - 38s - loss: 6.2220e-04 - acc: 1.000

42/42 [==============================] - 41s - loss: 0.0016 - acc: 1.0000 - val_loss: 4.2155e-04 - val_acc: 1.0000
Epoch 15/100
42/42 [==============================] - 41s - loss: 0.0015 - acc: 1.0000 - val_loss: 3.2255e-04 - val_acc: 1.0000
Epoch 16/100
42/42 [==============================] - 41s - loss: 0.0015 - acc: 1.0000 - val_loss: 2.6428e-04 - val_acc: 1.0000
Epoch 17/100
42/42 [==============================] - 40s - loss: 0.0016 - acc: 1.0000 - val_loss: 2.7679e-04 - val_acc: 1.0000
Epoch 18/100
42/42 [==============================] - 41s - loss: 0.0015 - acc: 1.0000 - val_loss: 3.1366e-04 - val_acc: 1.0000
Epoch 19/100
42/42 [==============================] - 38s - loss: 0.0015 - acc: 1.0000 - val_loss: 3.3492e-04 - val_acc: 1.0000
Epoch 20/100
42/42 [==============================] - 41s - loss: 0.0015 - acc: 1.0000 - val_loss: 3.3822e-04 - val_acc: 1.0000
Epoch 21/100
42/42 [==============================] - 41s - loss: 0.0015 - acc: 1.0000 - val_loss: 3.5472e-04 - val_a

In [ ]:
import resource